In [1]:
%load_ext autoreload
%autoreload 2
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, generate_demand, generate_vehicles, initialize_df  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate
from MaaSSim.visualizations import plot_veh
from MaaSSim.shared import prep_shared_rides
import logging
import matplotlib.pyplot as plt

import pandas as pd
import ExMAS

In [2]:
params = get_config('../../data/config/delft.json')  # load configuration

params.times.pickup_patience = 3600 # 1 hour of simulation
params.simTime = 4 # 6 minutes hour of simulation
params.nP = 100 # reuqests (and passengers)
params.nV = 10 # vehicles

In [3]:
params.t0 = pd.Timestamp.now()
params.shareability.avg_speed = params.speeds.ride
params.shareability.shared_discount = 0.25
params.shareability.delay_value = 1
params.shareability.WtS = 1.3
params.shareability.price = 1.5 #eur/km
params.shareability.VoT = 0.0035 #eur/s
params.shareability.matching_obj = 'u_veh' #minimize VHT for vehicles
params.shareability.pax_delay = 0
params.shareability.horizon = 600
params.shareability.max_degree = 4
params.shareability.nP = params.nP
params.shareability.share = 1
params.shareability.without_matching = True
params.shareability.operating_cost = 0.5
params.shareability.comm_rate = 0.2

inData = load_G(inData, params)  # load network graph 

inData = generate_demand(inData, params, avg_speed = False)
inData.vehicles = generate_vehicles(inData,params.nV)
inData.vehicles.platform = inData.vehicles.apply(lambda x: 0, axis = 1)
inData.passengers.platforms = inData.passengers.apply(lambda x: [0], axis = 1)
inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0], axis = 1) 
inData.platforms = initialize_df(inData.platforms)
inData.platforms.loc[0]=[1,'Uber',30]
params.shareability.share = 1
params.shareability.without_matching = True



In [4]:
inData = ExMAS.main(inData, params.shareability, plot=False) # create shareability graph (ExMAS) 

21-06-23 12:31:47-INFO-Initializing pairwise trip shareability between 100 and 100 trips.
21-06-23 12:31:47-INFO-creating combinations
21-06-23 12:31:47-INFO-9900	 nR*(nR-1)
21-06-23 12:31:47-INFO-Reduction of feasible pairs by 99.85%
21-06-23 12:31:47-INFO-Degree 2 	Completed
21-06-23 12:31:47-INFO-trips to extend at degree 2 : 43
21-06-23 12:31:47-INFO-At degree 2 feasible extensions found out of 0 searched
21-06-23 12:31:47-INFO-Degree 3 	Completed
21-06-23 12:31:47-INFO-No more trips to exted at degree 3


In [5]:
inData = prep_shared_rides(inData, params.shareability) # prepare schedules

21-06-23 12:31:47-INFO-Matching 100 trips to 143 rides in order to min u_veh
21-06-23 12:31:47-INFO-Problem solution: Optimal. 
Total costs for single trips:         24,885 
reduced by matching to:               23,286


In [6]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],7.837500,423,1,[7.8375],"[0, 423]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1615493865, 1570984309]","[None, 0, 0]",node time req_id od 0 ...,423,4.23,6.357000,1.271400,5.0856
1,[1],4.003500,216,1,[4.0035],"[5, 216]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1615493865, 626091907]","[None, 1, 1]",node time req_id od 0 ...,216,2.16,3.247500,0.649500,2.5980
2,[2],4.176000,225,1,[4.176],"[230, 225]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1410536856, 44848456]","[None, 2, 2]",node time req_id od 0 ...,225,2.25,3.388500,0.677700,2.7108
3,[3],4.096000,221,1,[4.096],"[365, 221]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1435362343, 44834060]","[None, 3, 3]",node time req_id od 0 ...,221,2.21,3.322500,0.664500,2.6580
4,[4],8.431000,455,1,[8.431],"[415, 455]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1435362459, 1402734949]","[None, 4, 4]",node time req_id od 0 ...,455,4.55,6.838500,1.367700,5.4708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,"[27, 28]",11.816325,609,21,"[8.2286, 3.587725]","[4023.5, 319, 199, 91]","[27, 28]","[28, 27]",2,138,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1668136717, 1552650442, 44842015, 42443...","[None, 27, 28, 28, 27]",node time req_id od 0 ...,609,6.09,7.270875,1.454175,5.8167
139,"[12, 14]",11.878625,523,21,"[6.920925, 4.9577]","[2190.5, 134, 296, 93]","[12, 14]","[14, 12]",2,139,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 107981525, 1584562931, 4564972422, 1530...","[None, 12, 14, 14, 12]",node time req_id od 0 ...,523,5.23,7.601625,1.520325,6.0813
140,"[0, 1]",10.151775,427,21,"[6.7219750000000005, 3.4298]","[2.5, 0, 216, 211]","[0, 1]","[1, 0]",2,140,...,"[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1615493865, 1615493865, 626091907, 1570...","[None, 0, 1, 1, 0]",node time req_id od 0 ...,427,4.27,7.203375,1.440675,5.7627
141,"[75, 76]",14.820850,690,21,"[9.411525000000001, 5.409325]","[10895.0, 126, 341, 223]","[75, 76]","[76, 75]",2,141,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44817455, 2588260101, 44823290, 44770870]","[None, 75, 76, 76, 75]",node time req_id od 0 ...,690,6.90,10.102500,2.020500,8.0820


In [7]:
responses = []
avg_kpi = []
idle_time = []

for i in range(1, 4):
    params.kpi = i
    sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate
    sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/veh{}.csv'.format(i))
    sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/pax{}.csv'.format(i))
    #driver_data.loc['Cost'] = driver_data.loc['Revenue'].apply(lambda x: x*params.shareability.operating_cost)['Vehicles'] = sim.res[0].veh_exp.index
    #sim.res[0].veh_exp['ds'] = f"{i}"
    
    #responses.append(sim.res[0].veh_exp)
     
   # vehicles = sim.res[0].veh_exp.loc[sim.res[0].veh_exp["nRIDES"] > 0]
    #no_of_veh = len(vehicles)
    
    #avg_kpi.append(sim.res[0].all_kpi/no_of_veh)
    #idle_time.append(vehicles['IDLE'].sum()/no_of_veh)
    
    #_inData.sblts.rides['commission'] = _inData.sblts.rides.apply(lambda row: row.fare*sp.comm_rate, axis=1)
    

21-06-23 12:31:48-INFO-Matching 100 trips to 143 rides in order to min u_veh
21-06-23 12:31:48-INFO-Problem solution: Optimal. 
Total costs for single trips:         24,885 
reduced by matching to:               23,286
0.5
21-06-23 12:31:49-CRITICAL-this is request 0 with [0, 119, 140] available rides.
21-06-23 12:31:49-CRITICAL-ride 0 available [1]
21-06-23 12:31:49-CRITICAL-ride 119 available [0, 1]
21-06-23 12:31:49-CRITICAL-ride 140 available [1, 0]
21-06-23 12:31:49-CRITICAL-this is reuqest 0 with [0, 119, 140] still available rides.
Profit Maximization
21-06-23 12:31:49-CRITICAL-vehicle 1 has 3 choices
0.5
21-06-23 12:31:49-CRITICAL-this is request 1 with [1, 119, 140] available rides.
21-06-23 12:31:49-CRITICAL-ride 1 available [1]
21-06-23 12:31:49-CRITICAL-ride 119 not available [1, 2]
21-06-23 12:31:49-CRITICAL-ride 140 not available [2, 1]
21-06-23 12:31:49-CRITICAL-this is reuqest 1 with [1] still available rides.
0.5
0.5
21-06-23 12:31:49-CRITICAL-this is request 3 with [3